# 12. Scikit-Learn Machine Learning Pipeline

## Required steps
### Pre-process
    - [X] Categorical to Numeric
    - [X] Clean Numeric Data
    - [X] Outlier Detection
    - [X] Data Transformation
    - [X] Data Normalization
    - [X] Dimensionality Reduction
    
### Training steps - models
    - [X] Kernel Ridge
    - [X] XGBoost Regression
    - [X] Lasso Regression
    - [X] ANN Regression
    - [X] Random Forest Regression
    - [X] ElasticNet
    - [X] Bayesian Ridge Regression
    - [X] LassoLarsIC Regression
    - [X] Gradient Boosting Regression
    
### Stacking
    - [ ] Picking one of the traning models as Meta-model
    - [ ] Use the rest for training the evaluation set and test on the test set.
    - [ ] Use Meta-model to predict the test set based on the trained models results.
    
### Ensembling [?]

---------------------

Connect to Google Drive - mahdi.shooshtari@gmail.com

In [1]:
from google.colab import drive
drive.mount('/content/drive')
main_folder = '/content/drive/My Drive/Kaggle_House_Prices'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


---------------

## Implementation
## Pre-process classes definition

In [0]:
import numpy as np
import pandas as pd

from sklearn.base import BaseEstimator, TransformerMixin

### Convert Categorical columns to Numeric

In [3]:
! python -m pip install --upgrade pip
! pip install category_encoders
import json 
import category_encoders as ce
# categorical missing value imputer
class CatToNum(BaseEstimator, TransformerMixin):

    def __init__(self, dict_address, continuesVars, descreteVars_Ordinal, descreteVars_Nominal):
        self.continuesVars = continuesVars
        self.descreteVars_Ordinal = descreteVars_Ordinal
        self.descreteVars_Nominal = descreteVars_Nominal
        
        fileName = dict_address
        f = open(fileName,'r')
        self.conversion_dict = json.loads(f.read())
        

    def fit(self, X, y=None):
        # we need the fit statement to accomodate the sklearn pipeline
        des_nom_DF = X[self.descreteVars_Nominal]
        # Map Nominal Categorical data to Numerical
        cat_nom_DF = des_nom_DF.fillna('NULL').astype(str)
        self.ce_binary = ce.BinaryEncoder()
        self.ce_binary.fit(cat_nom_DF)
        return self

    def transform(self, X):
        totalDF = X.copy()
        conDF = X[self.continuesVars]
        des_ord_DF = X[self.descreteVars_Ordinal]
        des_nom_DF = X[self.descreteVars_Nominal]
        
        # Map Ordinal Categorical data to Numerical
        cat_ord_DF_numerical = des_ord_DF.copy()
        for feature in self.conversion_dict:
            temp_dict = self.conversion_dict[feature]
            if ('NA' in temp_dict): # Replace 'NA' with np.nan
                temp_dict[np.nan] = temp_dict.pop('NA')
            cat_ord_DF_numerical[feature] = des_ord_DF[feature].map(temp_dict)
            
        totalDF[self.descreteVars_Ordinal] = cat_ord_DF_numerical
            
        # Map Nominal Categorical data to Numerical
        cat_nom_DF = des_nom_DF.fillna('NULL').astype(str)
        cat_nom_DF_numerical = self.ce_binary.transform(cat_nom_DF)
        totalDF = pd.concat([totalDF, cat_nom_DF_numerical], axis=1)
        totalDF.drop(self.descreteVars_Nominal, axis=1, inplace=True)
        cols_to_drop = [x for x in list(cat_nom_DF_numerical) if ('_0' in x)]
        totalDF.drop(cols_to_drop, axis=1, inplace=True)

        return totalDF


Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (19.3.1)


### Clean Numeric Data

In [0]:
class CleanNum(BaseEstimator, TransformerMixin):

    def __init__(self, measure='mean', drop_thresh=0.8):
        self.measure = measure
        self.drop_thresh = drop_thresh

    def fit(self, X, y=None):
        train_df = X.copy().astype(float)
        # Remove columns with dominance bigger than a threshold
        self.drop_list = []
        for feature in train_df:
            col_df = train_df[feature]
            count_nan = col_df.isnull().sum()
            nan_ratio = count_nan/len(col_df)   
            repeats = train_df.pivot_table(index=[feature], aggfunc='size').sort_values()
            max_repeat_ratio = repeats.max()/len(col_df)
            if (nan_ratio>self.drop_thresh or max_repeat_ratio>self.drop_thresh):
                self.drop_list.append(feature)

        # Replace null values with average (or mode) of the train columns
        self.cols_average = train_df.mean(axis = 0)
        self.cols_mode = train_df.mode(axis = 0)
        
        return self

    def transform(self, X):
        totalDF = X.copy().astype(float)
        
        # Replace null values with average (or mode) of the train columns
        for col in totalDF:
            if (self.measure=='mean'):
                totalDF[col] = totalDF[col].fillna(self.cols_average[col])
            if (self.measure=='mode'):
                totalDF[col] = totalDF[col].fillna(self.cols_mode[col])
        
        # Remove columns with dominance bigger than a threshold
        totalDF.drop(self.drop_list, axis=1, inplace=True)

        return totalDF

### Outlier Detection

In [0]:
class OutlierDetection(BaseEstimator, TransformerMixin):

    def __init__(self):
        pass

        # Function to Detection Outlier on one-dimentional datasets.
    def find_anomalies(self,data):
        anomalies_idx = []
        # Set upper and lower limit to 3 standard deviation
        data_std = data.std()
        data_mean = data.mean()
        anomaly_cut_off = data_std * 3

        lower_limit  = data_mean - anomaly_cut_off 
        upper_limit = data_mean + anomaly_cut_off
        # Generate outliers
        for idx in range(len(data)):
            outlier = data[idx]
            if outlier > upper_limit or outlier < lower_limit:
                anomalies_idx.append(idx)
        return anomalies_idx

    def fit(self, X, y=None):
        train_df = X.copy()
        outliers_full_list = []
        for feature in train_df:
            outliers_full_list = outliers_full_list + self.find_anomalies(train_df[feature].values)

        self.outliers_unique = list(set(int(outliers_full_list)))
        return self

    def transform(self, X):
        totalDF = X.copy()
        totalDF.drop(self.outliers_unique, inplace=True)
        return totalDF

### Data Transformation (Box-Cox transformation)

In [0]:
from scipy.special import boxcox1p
from scipy.stats import norm, skew

class DataTransformation(BaseEstimator, TransformerMixin):

    def __init__(self):
        pass
    
    def fit(self, X, y=None, thresh=0.75, lam=0.15):
        self.lam = lam
        train_df = X.copy()
        # Check the skew of all numerical features
#         print(type(skew(train_df)))
        skewed_feats = train_df.apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
#         print(type(skewed_feats ))
        skewness = pd.DataFrame({'Skew' :skewed_feats})
        skewness = skewness[abs(skewness) > thresh]
        self.skewed_features_idx_list = skewness.index
        return self
    
    def transform(self, X):
        totalDF = X.copy()
#         skewed_features = totalDF[self.skewed_features_list]
        for feat_idx in self.skewed_features_idx_list:
            #all_data[feat] += 1
            totalDF[feat_idx] = boxcox1p(totalDF[feat_idx], self.lam)
        return totalDF

### Data Normalization
There is no need to define a specific class for data normalization.

### Dimensionality Reduction
I am thinking there is no need for dimensionality reduction at the moment. But I may change my mind later...

----

### Artificial Neural Networks Class definition

In [7]:
import keras
from keras.models import Sequential
from keras.layers import Dense

class ANN(BaseEstimator, TransformerMixin):

    def __init__(self, optimizer='adam', loss='mean_squared_error', batch_size=100, nb_epoch=2000):
        self.optimizer = optimizer
        self.loss = loss
        self.batch_size = batch_size
        self.nb_epoch = nb_epoch
    
    def fit(self, X, y=None):
        regressor = Sequential()
        # Adding the input layer and the first hidden layer 
        # Number of nodes in each layer: Average of number of input nodes and ouput nodes
        # In our case, we have 42 input nodes and 2 classes, therfore 1 output node, so the average is 22 or 21.
        (rows, cols) = X.shape
        numberOfInputNodes = cols
        numberOfOutputNodes = 1
        numberOfMiddleNodes = int((numberOfInputNodes + numberOfOutputNodes)/2)
        regressor.add(Dense(input_dim=numberOfInputNodes, output_dim=numberOfMiddleNodes, activation='relu'))
        regressor.add(Dense(output_dim=numberOfMiddleNodes, activation='relu'))
        regressor.add(Dense(output_dim=numberOfMiddleNodes, activation='relu'))
        regressor.add(Dense(output_dim=numberOfOutputNodes, activation='linear'))

        # Compiling the ANN
        regressor.compile(optimizer=self.optimizer, loss=self.loss)
        # Fitting the ANN to the training set
        regressor.fit(X, y, batch_size=self.batch_size, nb_epoch=self.nb_epoch)
        self.regressor = regressor
        
        return self
    
    def transform(self, X):
        pass
    
    def predict(self, X):
      return self.regressor.predict(X)


Using TensorFlow backend.


-----------------------------------

In [0]:
n_folds = 5

def rmsle_cv(model, train, y_train):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

------

## Training steps - models

In [0]:
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_squared_error

In [0]:
train_file = main_folder+'/house-prices-advanced-regression-techniques/train.csv'
train_df = pd.read_csv(train_file)
test_file = main_folder+'/house-prices-advanced-regression-techniques/test.csv'
test_df = pd.read_csv(test_file)





descreteVars_Nominal = ['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LandContour', 'LotConfig', 'Neighborhood',
                        'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
                        'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating', 'CentralAir', 'Electrical',
                        'GarageType', 'MiscFeature', 'SaleType', 'SaleCondition']

descreteVars_Ordinal = ['LotShape', 'Utilities', 'LandSlope', 'ExterQual', 'ExterCond', 'BsmtQual',
                        'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC',
                        'KitchenQual','Functional',  'FireplaceQu', 'GarageFinish', 'GarageQual', 'GarageCond',
                        'PavedDrive', 'PoolQC', 'Fence']

resultVar = ['SalePrice']



continuesVars = list(set(list(train_df)) - set(descreteVars_Nominal) - set(descreteVars_Ordinal) - set(resultVar))

dict_address= main_folder+'/cat_ord_dict.txt'


X = train_df.drop(resultVar, axis=1)
y = train_df[resultVar]


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

In [0]:
models_names = ['KernelRidge', 'ElasticNet', 'Lasso', 'GradientBoostingRegressor', 'BayesianRidge', 'LassoLarsIC', 'RandomForestRegressor', 'XGBRegressor', 'ANN']
models_list = [KernelRidge(), ElasticNet(), Lasso(), GradientBoostingRegressor(), BayesianRidge(), LassoLarsIC(), RandomForestRegressor(), XGBRegressor(), ANN()]


KR_param_grid = {'alpha': 0.1, 'coef0': 100, 'degree': 1, 'gamma': None, 'kernel': 'polynomial'}
EN_param_grid = {'alpha': 0.001, 'copy_X': True, 'l1_ratio': 0.6, 'fit_intercept': True, 'normalize': False, 
                         'precompute': False, 'max_iter': 300, 'tol': 0.001, 'selection': 'random', 'random_state': None}
LASS_param_grid = {'alpha': 0.0005, 'copy_X': True, 'fit_intercept': True, 'normalize': False, 'precompute': False, 
                    'max_iter': 300, 'tol': 0.01, 'selection': 'random', 'random_state': None}
GB_param_grid = {'loss': 'huber', 'learning_rate': 0.1, 'n_estimators': 300, 'max_depth': 3, 
                                        'min_samples_split': 0.0025, 'min_samples_leaf': 5}
BR_param_grid = {'n_iter': 200, 'tol': 0.00001, 'alpha_1': 0.00000001, 'alpha_2': 0.000005, 'lambda_1': 0.000005, 
                 'lambda_2': 0.00000001, 'copy_X': True}
LL_param_grid = {'criterion': 'aic', 'normalize': True, 'max_iter': 100, 'copy_X': True, 'precompute': 'auto', 'eps': 0.000001}
RFR_param_grid = {'n_estimators': 50, 'max_features': 'auto', 'max_depth': None, 'min_samples_split': 5, 'min_samples_leaf': 2}
XGB_param_grid = {'max_depth': 3, 'learning_rate': 0.1, 'n_estimators': 300, 'booster': 'gbtree', 'gamma': 0, 'reg_alpha': 0.1,
                  'reg_lambda': 0.7, 'max_delta_step': 0, 'min_child_weight': 1, 'colsample_bytree': 0.5, 'colsample_bylevel': 0.2,
                  'scale_pos_weight': 1}
ANN_param_grid = {'optimizer': 'adam', 'loss': 'mean_squared_error'}
params_grid = [KR_param_grid, EN_param_grid, LASS_param_grid, GB_param_grid, BR_param_grid, LL_param_grid, RFR_param_grid, XGB_param_grid, ANN_param_grid]

In [0]:
def cal_rmse(model, X, y):
    return (np.sqrt(mean_squared_error(y, model.predict(X))))

In [0]:
original_test_file = main_folder+'/house-prices-advanced-regression-techniques/test.csv'
original_test_df = pd.read_csv(original_test_file)
id_col = original_test_df['Id']


y_val_target = pd.DataFrame(y_val, columns=['SalePrice'])
y_val_target.to_csv(main_folder+'/Results/y_val_target.csv', index=False)

In [13]:


for i in range(len(models_list)):
    print(models_names[i])
    model = models_list[i]
    params = params_grid[i]
    print(params)
    pipeline = make_pipeline(CatToNum(dict_address, continuesVars, descreteVars_Ordinal, descreteVars_Nominal), CleanNum(), RobustScaler(), model.set_params(**params))
    pipeline_transformed = make_pipeline(CatToNum(dict_address, continuesVars, descreteVars_Ordinal, descreteVars_Nominal), CleanNum(), DataTransformation(), RobustScaler(), model.set_params(**params))

    pipeline.fit(X_train, y_train)    
    pipeline_transformed.fit(X_train, y_train)    
    
    print('Normal ' + models_names[i])
    print('Train rmse: ')
    print(cal_rmse(pipeline, X_train, y_train))
    print('Validation rmse: ')
    print(cal_rmse(pipeline, X_val, y_val))
    print(models_names[i]+' with transformed skewed columns')
    print('Train rmse: ')
    print(cal_rmse(pipeline_transformed, X_train, y_train))
    print('Validation rmse: ')
    print(cal_rmse(pipeline_transformed, X_val, y_val))
    print('-----------------------------------------------------')
    
    #####################Save validation results#######################
    Y_pred = pipeline.predict(X_val)
    result_df = pd.DataFrame(Y_pred, columns=['SalePrice'])
    result_df.to_csv(main_folder+'/Results/'+models_names[i]+'_Val.csv', index=False)
    
    Y_pred_transformed = pipeline_transformed.predict(X_val)
    result_df = pd.DataFrame(Y_pred_transformed, columns=['SalePrice'])
    result_df.to_csv(main_folder+'/Results/'+models_names[i]+'_transformed_Val.csv', index=False)

    #####################Save test results#######################
    Y_pred = pipeline.predict(test_df)
    result_df = pd.concat([id_col, pd.DataFrame(Y_pred, columns=['SalePrice'])], axis=1)
    result_df.to_csv(main_folder+'/Results/'+models_names[i]+'_Test.csv', index=False)
    
    
    Y_pred_transformed = pipeline_transformed.predict(test_df)
    result_df = pd.concat([id_col, pd.DataFrame(Y_pred_transformed, columns=['SalePrice'])], axis=1)
    result_df.to_csv(main_folder+'/Results/'+models_names[i]+'_transformed_Test.csv', index=False)
    
    


KernelRidge
{'alpha': 0.1, 'coef0': 100, 'degree': 1, 'gamma': None, 'kernel': 'polynomial'}
Normal KernelRidge
Train rmse: 
40477.34158385059
Validation rmse: 
33752.360766792684
KernelRidge with transformed skewed columns
Train rmse: 
33964.609106315176
Validation rmse: 
35648.87117435537
-----------------------------------------------------
ElasticNet
{'alpha': 0.001, 'copy_X': True, 'l1_ratio': 0.6, 'fit_intercept': True, 'normalize': False, 'precompute': False, 'max_iter': 300, 'tol': 0.001, 'selection': 'random', 'random_state': None}


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 527140957668.0031, tolerance: 6150984128.205524
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 588230248717.4498, tolerance: 6150984128.205524
  positive)


Normal ElasticNet
Train rmse: 
41326.69431257437
Validation rmse: 
34811.22302637462
ElasticNet with transformed skewed columns
Train rmse: 
33899.304679275854
Validation rmse: 
35853.758192109424
-----------------------------------------------------
Lasso
{'alpha': 0.0005, 'copy_X': True, 'fit_intercept': True, 'normalize': False, 'precompute': False, 'max_iter': 300, 'tol': 0.01, 'selection': 'random', 'random_state': None}


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 526255602961.3499, tolerance: 61509841282.055244
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 587032631668.0398, tolerance: 61509841282.055244
  positive)


Normal Lasso
Train rmse: 
41410.74739587303
Validation rmse: 
34920.54207288292
Lasso with transformed skewed columns
Train rmse: 
33898.35125167885
Validation rmse: 
35884.99037634774
-----------------------------------------------------
GradientBoostingRegressor
{'loss': 'huber', 'learning_rate': 0.1, 'n_estimators': 300, 'max_depth': 3, 'min_samples_split': 0.0025, 'min_samples_leaf': 5}


...

Normal RandomForestRegressor
Train rmse: 
34016.030458393776
Validation rmse: 
34892.48396355968
RandomForestRegressor with transformed skewed columns
Train rmse: 
14780.712862474955
Validation rmse: 
26782.766404239603
-----------------------------------------------------
XGBRegressor
{'max_depth': 3, 'learning_rate': 0.1, 'n_estimators': 300, 'booster': 'gbtree', 'gamma': 0, 'reg_alpha': 0.1, 'reg_lambda': 0.7, 'max_delta_step': 0, 'min_child_weight': 1, 'colsample_bytree': 0.5, 'colsample_bylevel': 0.2, 'scale_pos_weight': 1}
[22:08:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:08:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Normal XGBRegressor
Train rmse: 
31068.919521136166
Validation rmse: 
35756.92039399908
XGBRegressor with transformed skewed columns
Train rmse: 
10877.737541056262
Validation rmse: 
24250.32936816531
----------

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=61, activation="relu", units=31)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=31)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=31)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="linear", units=1)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.




Epoch 1/2000





1022/1022 [==============================] - 0s 241us/step - loss: 38893014647.2329
Epoch 2/2000
1022/1022 [==============================] - 0s 17us/step - loss: 38892778994.9746
Epoch 3/2000
1022/1022 [==============================] - 0s 19us/step - loss: 38892509087.8121
Epoch 4/2000
1022/1022 [==============================] - 0s 21us/step - loss: 38892096429.8395
Epoch 5/2000
1022/1022 [==============================] - 0s 18us/step - loss: 38891349234.4736
Epoch 6/2000
1022/1022 [==============================] - 0s 19us/step - loss: 38889913903.0920
Epoch 7/2000
1022/1022 [==============================] - 0s 23us/step - loss: 38887120896.0000
Epoch 8/2000
1022/1022 [==============================] - 0s 17us/step - loss: 38882081128.7045
Epoch 9/2000
1022/1022 [==============================] - 0s 24us/step - loss: 38873352684.9628
Epoch 10/2000
1022/1022 [==============================] - 0s 19us/step - loss: 38859039413.3542
Epoch 11/2000
1022/1022 [======

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=61, activation="relu", units=31)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=31)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=31)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="linear", units=1)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/2000
1022/1022 [==============================] - 0s 250us/step - loss: 38892981558.6067
Epoch 2/2000
1022/1022 [==============================] - 0s 21us/step - loss: 38892786569.7691
Epoch 3/2000
1022/1022 [==============================] - 0s 21us/step - loss: 38892575168.8767
Epoch 4/2000
1022/1022 [==============================] - 0s 21us/step - loss: 38892181812.6027
Epoch 5/2000
1022/1022 [==============================] - 0s 19us/step - loss: 38891289557.9178
Epoch 6/2000
1022/1022 [==============================] - 0s 19us/step - loss: 38889396268.0861
Epoch 7/2000
1022/1022 [==============================] - 0s 19us/step - loss: 38885596606.8728
Epoch 8/2000
1022/1022 [==============================] - 0s 24us/step - loss: 38878372092.4932
Epoch 9/2000
1022/1022 [==============================] - 0s 20us/step - loss: 38865420664.7358
Epoch 10/2000
1022/1022 [==============================] - 0s 24us/step - loss: 38843614017.6282
Epoch 11/2000
1022/1022 [=============

## Stacking

### Picking meta models

In [27]:
best_models = ['GradientBoostingRegressor', 'BayesianRidge', 'RandomForestRegressor', 'XGBRegressor', 'ANN']
best_models_idx = [3, 4, 6, 7, 8]
best_models_transformed = [True, False, True, True, True]
for i in range(len(best_models_idx)):
  model_idx = best_models_idx[i]
  model_name = models_names[model_idx]
  model = models_list[model_idx]
  transformed = best_models_transformed[i]
  print(model_name)
  print('Transformed: '+str(transformed))

GradientBoostingRegressor
Transformed: True
BayesianRidge
Transformed: False
RandomForestRegressor
Transformed: True
XGBRegressor
Transformed: True
ANN
Transformed: True


### Reading validation results as the training data

In [26]:
import os 
from glob import glob
result_folder = main_folder+'/Results'
val_files_list = glob(result_folder+'/*_Val.csv')
total_val_list = []
for val_file in val_files_list:
  filename = os.path.basename(val_file).replace('_Val.csv', '')
  temp_df = pd.read_csv(val_file)
  temp_df.columns = ['SalePrice_'+filename]
  total_val_list.append(temp_df)

y_val_target = pd.read_csv(main_folder+'/Results/y_val_target.csv')
total_val_list.append(y_val_target)
total_val_df = pd.concat(total_val_list, axis=1)
total_val_df.to_csv('Total_val.csv', index=False)
total_val_df.to_csv(main_folder+'/Results/Total_val.csv', index=False)


     SalePrice
0       154500
1       325000
2       115000
3       159000
4       315500
..         ...
433     139000
434     126175
435     205950
436     110000
437     485000

[438 rows x 1 columns]
     SalePrice_KernelRidge  ...  SalePrice
0            151543.130803  ...     154500
1            339148.678221  ...     325000
2            128235.351923  ...     115000
3            182690.646940  ...     159000
4            340090.981990  ...     315500
..                     ...  ...        ...
433          122256.492559  ...     139000
434           96827.376724  ...     126175
435          217863.077382  ...     205950
436          152503.077301  ...     110000
437          464922.680878  ...     485000

[438 rows x 19 columns]


### Reading test results as the test data

In [21]:
import os 
from glob import glob
result_folder = main_folder+'/Results'
test_files_list = glob(result_folder+'/*_Test.csv')
total_test_list = []
for test_file in test_files_list:
  filename = os.path.basename(test_file).replace('_Test.csv', '')
  print(filename)
  temp_df = pd.read_csv(test_file)
  temp_df.drop(['Id'], axis=1, inplace=True)
  temp_df.columns = ['SalePrice_'+filename]
  total_test_list.append(temp_df)

total_test_df = pd.concat(total_test_list, axis=1)
total_test_df.to_csv('Total_test.csv', index=False)
total_test_df.to_csv(main_folder+'/Results/Total_test.csv', index=False)


KernelRidge
KernelRidge_transformed
ElasticNet
ElasticNet_transformed
Lasso
Lasso_transformed
GradientBoostingRegressor
GradientBoostingRegressor_transformed
BayesianRidge
BayesianRidge_transformed
LassoLarsIC
LassoLarsIC_transformed
RandomForestRegressor
RandomForestRegressor_transformed
XGBRegressor
XGBRegressor_transformed
ANN
ANN_transformed
      SalePrice_KernelRidge  ...  SalePrice_ANN_transformed
0             103169.170236  ...                  112012.40
1             167248.568661  ...                  162064.05
2             188103.584955  ...                  184807.31
3             185211.617740  ...                  191475.47
4             189548.498901  ...                  182958.23
...                     ...  ...                        ...
1454           55374.277379  ...                   78412.19
1455           67693.650125  ...                   83586.32
1456          186147.130144  ...                  186714.81
1457           98169.430190  ...                  11

### Train the meta model on the valid set and preidct the test set

In [42]:
original_test_file = main_folder+'/house-prices-advanced-regression-techniques/test.csv'
original_test_df = pd.read_csv(original_test_file)
id_col = original_test_df['Id']

best_models = ['GradientBoostingRegressor', 'BayesianRidge', 'RandomForestRegressor', 'XGBRegressor', 'ANN']
best_models_idx = [3, 4, 6, 7, 8]
best_models_transformed = [True, False, True, True, True]

X_train = total_val_df.drop(['SalePrice'], axis=1)
y_train = total_val_df['SalePrice']

X_test = total_test_df


for i in range(len(best_models_idx)):
  model_idx = best_models_idx[i]
  model_name = models_names[model_idx]
  model = models_list[model_idx]
  params = params_grid[model_idx]


  transformed = best_models_transformed[i]
  col_to_drop = 'SalePrice_'+model_name
  if (transformed):
    col_to_drop += '_transformed'
  
  temp_X_train = X_train.drop([col_to_drop], axis=1)
  temp_X_test = X_test.drop([col_to_drop], axis=1)

  pipeline = make_pipeline(RobustScaler(), model.set_params(**params))
  

  pipeline.fit(temp_X_train, y_train) 
  

  print('Train rmse: ')
  print(cal_rmse(pipeline, temp_X_train, y_train))

  y_pred = pipeline.predict(temp_X_test)   

  
  result_df = pd.concat([id_col, pd.DataFrame(y_pred, columns=['SalePrice'])], axis=1)
  result_df.to_csv(model_name+'_final.csv', index=False)

  

  # print(y_pred.shape)
  # print(model)

Train rmse: 
10676.977265018937
Train rmse: 
22304.695890045434
Train rmse: 
12837.531077401372
[23:48:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Train rmse: 
7199.608048853281


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=17, activation="relu", units=9)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=9)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=9)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Update your `Dense` call to the Keras 2

Epoch 1/2000
438/438 [==============================] - 1s 1ms/step - loss: 39381144487.1598
Epoch 2/2000
438/438 [==============================] - 0s 28us/step - loss: 39381081466.7397
Epoch 3/2000
438/438 [==============================] - 0s 25us/step - loss: 39381024908.2740
Epoch 4/2000
438/438 [==============================] - 0s 24us/step - loss: 39380967741.9543
Epoch 5/2000
438/438 [==============================] - 0s 25us/step - loss: 39380916887.9635
Epoch 6/2000
438/438 [==============================] - 0s 24us/step - loss: 39380866744.6941
Epoch 7/2000
438/438 [==============================] - 0s 23us/step - loss: 39380816246.0639
Epoch 8/2000
438/438 [==============================] - 0s 28us/step - loss: 39380769431.9635
Epoch 9/2000
438/438 [==============================] - 0s 28us/step - loss: 39380718559.2694
Epoch 10/2000
438/438 [==============================] - 0s 27us/step - loss: 39380666171.6164
Epoch 11/2000
438/438 [==============================] - 0s 